# Shor Generic with Simulator
Works for:
- 15 -> 0.3 sec
- 21 -> 1.1 sec
- 35 -> 12.5 sec
- 51 -> 47 sec
- 77 -> NO > 10min

In [ ]:
import time
import qiskit
from qiskit import QuantumCircuit, transpile, QuantumRegister, ClassicalRegister
from qiskit_aer import AerSimulator
import math
import random
from fractions import Fraction
from qiskit.circuit.library import QFT
import numpy as np

def c_amodN_generic(a, N, power_of_2, num_qubits_val):
    """
    Crea l'operatore Controlled-U_a^(2^k) mod N utilizzando un approccio basato su matrice.
    Questo è adatto per N piccoli ma non è scalabile per N grandi.

    Args:
        a (int): La base per l'elevamento a potenza.
        N (int): Il modulo.
        power_of_2 (int): La potenza di 2 (2^k) per a^(2^k).
        num_qubits_val (int): Numero di qubit nel registro dei valori (per rappresentare N).

    Returns:
        ControlledGate: Un gate controllato che esegue l'operazione.
        None: Se si trova un fattore non banale classicamente.
    """
    effective_a = pow(a, power_of_2, N)

    # L'operatore di moltiplicazione modulare U_x|y> = |(x*y) mod N> è unitario solo se x è coprimo con N.
    common_divisor = math.gcd(effective_a, N)
    if common_divisor != 1:
        print(f"!!! ATTENZIONE: Found common divisor {common_divisor} with effective_a={effective_a} and N={N}.")
        print(f"!!! Questo significa che {common_divisor} è un fattore non banale di {N} (trovato classicamente).")
        raise ValueError(f"Non-unitary U_gate detected due to gcd({effective_a},{N})={common_divisor} != 1. Factor found: {common_divisor}")

    # 2. Crea la matrice di permutazione per l'operazione.
    # La dimensione della matrice è 2^n x 2^n, dove n = num_qubits_val.
    dim = 2**num_qubits_val
    U_matrix = np.zeros((dim, dim), dtype=complex)

    # La mappatura U_a|y> = |(a*y) mod N> è una permutazione solo sui valori y < N.
    # Per rendere l'operatore unitario sull'intero spazio {0...2^n-1},
    # si definisce che agisca come identità per y >= N.
    for y_in in range(dim):
        if y_in < N:
            # Applica la moltiplicazione modulare per gli stati nel dominio di N
            y_out = (effective_a * y_in) % N
            U_matrix[y_out][y_in] = 1
        else:
            # Lascia invariati gli stati al di fuori del dominio di N (mappatura identità)
            # Questo garantisce che la matrice complessiva sia una matrice di permutazione e quindi unitaria.
            y_out = y_in
            U_matrix[y_out][y_in] = 1

    # 3. Converti la matrice in un gate controllato di Qiskit.
    U_gate = qiskit.quantum_info.Operator(U_matrix).to_instruction()
    U_gate.label = f"U_{effective_a}_mod_{N}"

    # 4. Crea il gate controllato da un singolo qubit.
    controlled_U_gate = U_gate.control(1)
    
    return controlled_U_gate

def build_shor_circuit(N):
    if N % 2 == 0 or N == 1:
        print("L'algoritmo di Shor non funziona per numeri pari (eccetto 2) o 1.")
        return []

    if math.isqrt(N) * math.isqrt(N) == N:
        print(f"N={N} è un quadrato perfetto. L'algoritmo di Shor non è necessario. Fattore: {math.isqrt(N)}")
        return [math.isqrt(N)]

    found_a_coprime = False
    a = -1
    for _ in range(20): 
        a = random.randint(2, N - 1)
        if math.gcd(a, N) == 1:
            found_a_coprime = True
            break
    if not found_a_coprime:
        print(f"Impossibile trovare un 'a' coprimo con N={N} dopo 20 tentativi.")
        return []
    print(f"Tentativo con N = {N}, scelto a = {a}")

    # Determina il numero di qubit
    n_qubit_function = N.bit_length()

    n_qubit_phase = 2 * n_qubit_function + 1 # Un qubit extra per maggiore precisione

    print(f"Qubit per funzione (n): {n_qubit_function}")
    print(f"Qubit per fase (t): {n_qubit_phase}")

    q_phase = QuantumRegister(n_qubit_phase, 'q_phase')
    q_function = QuantumRegister(n_qubit_function, 'q_function')
    c_bits = ClassicalRegister(n_qubit_phase, 'c_bits')

    qc = QuantumCircuit(q_phase, q_function, c_bits)

    qc.h(q_phase)
    qc.x(q_function[0]) # Inizializza il registro della funzione a |1> (00...01)

    for x in range(n_qubit_phase):
        power_of_2 = 2**x
        try:
            controlled_U_gate = c_amodN_generic(a, N, power_of_2, n_qubit_function)
        except ValueError as e:
            print(f"Errore nella creazione del gate U. L'algoritmo di Shor può finire qui classicamente. {e}")
            return []


        qc.append(controlled_U_gate, [q_phase[x]] + q_function[:])

    qc.append(QFT(n_qubit_phase).inverse(), q_phase)
    qc.measure(q_phase, c_bits)
    return qc, n_qubit_phase, a

def retrieve_factors(counts, n_qubit_phase, N, a):
    print("\nAnalisi dei risultati per trovare il periodo 'r':")
    found_factors = set()
    for measured_phase_str in counts:
        decimal_phase = int(measured_phase_str, 2) / (2**n_qubit_phase)
        if decimal_phase == 0:
            continue

        print(f"\nFase misurata (binario): {measured_phase_str}, (decimale): {decimal_phase}")

        frac = Fraction(decimal_phase).limit_denominator(N)
        s = frac.numerator
        r_candidate = frac.denominator

        print(f"Frazione approssimata: {s}/{r_candidate}")

        if r_candidate % 2 == 0:
            r_half = r_candidate // 2
            val_mod_N = pow(a, r_half, N)

            if val_mod_N != 1 and val_mod_N != (N - 1):
                print(f"Periodo r candidato valido: {r_candidate}")

                factor1 = math.gcd(pow(a, r_half) - 1, N)
                factor2 = math.gcd(pow(a, r_half) + 1, N)

                if factor1 != 1 and factor1 != N:
                    found_factors.add(factor1)
                    print(f"Fattore trovato: {factor1}")
                if factor2 != 1 and factor2 != N:
                    found_factors.add(factor2)
                    print(f"Fattore trovato: {factor2}")

                if len(found_factors) == 2 and (1 not in found_factors and N not in found_factors):
                    print(f"\n***** Trovati entrambi i fattori per N={N}: {list(found_factors)} *****")
                    return list(found_factors)
            else:
                print(f"Periodo r={r_candidate} non soddisfa la condizione a^(r/2) != +/- 1 mod N")
        else:
            print(f"Periodo r={r_candidate} è dispari, scartato.")


def run_shor_generic(N):
    qc, n_qubit_phase, a = build_shor_circuit(N)

    simulator = AerSimulator()
    compiled_circuit = transpile(qc, simulator)
    job = simulator.run(compiled_circuit, shots=1024)
    result = job.result()
    counts = result.get_counts(compiled_circuit)

    print("\nRisultati delle misurazioni (fasi):")
    print(counts)

    found_factors=retrieve_factors(counts, n_qubit_phase, N, a)
    
    if not found_factors:
        print(f"\nNon sono stati trovati fattori non banali per N={N} con a={a}.")
        print("Potrebbe essere necessario riprovare con un altro valore di 'a' o aumentare 'n_qubit_phase'.")
    else:
        print(f"\nFattori trovati (potrebbe essere incompleto): {list(found_factors)}")

    return list(found_factors)


# Test con N=15
print("\n***** Esecuzione per N=15 *****")
t=time.time()
run_shor_generic(15)
print(f"\nTempo impiegato: {time.time()-t}")

# Test con N=21
print("\n***** Esecuzione per N=21 *****")
t=time.time()
run_shor_generic(21)
print(f"\nTempo impiegato: {time.time()-t}")

# Test con N=35
print("\n***** Esecuzione per N=35 *****")
t=time.time()
run_shor_generic(35)
print(f"\nTempo impiegato: {time.time()-t}")

# Test con N=51 (3*17)
print("\n***** Esecuzione per N=51 *****")
t=time.time()
run_shor_generic(51)
print(f"\nTempo impiegato: {time.time()-t}")


# Test con N=77 (7*11) - n_qubit_function = 7 (2^7 = 128 stati)
print("\n***** Esecuzione per N=77 *****")
t=time.time()
run_shor_generic(77)
print(f"\nTempo impiegato: {time.time()-t}")


In [ ]:
import os
token=os.environ.get('QUANTUM_TOKEN')
backend_name="ibm_fex"
instance=os.environ.get('QUANTUM_INSTANCE')

# Shor Generic with Real Hardware
Works for:
- 15 -> 18 sec -> bit noisy
- 21 -> 2 min -> noisy, long traspilation

In [ ]:
import time
from qiskit import transpile
from qiskit_ibm_runtime import QiskitRuntimeService, Sampler



def run_shor_generic(N):
    
    qc, n_qubit_phase, a = build_shor_circuit(N)


    service = QiskitRuntimeService(channel="ibm_cloud", token=token, instance=instance)
    backend = service.backend(backend_name)
            
    compiled_circuit = transpile(qc, backend,optimization_level=3,
                seed_transpiler=42)

    print("Transpilazione completata.")
    sampler = Sampler(backend)
    # opt = sampler.options
    # opt.twirling.enable_gates = True         # attiva gate twirling
    # opt.twirling.enable_measure = True       # twirling anche sulle misure
    # opt.dynamical_decoupling.enable = True   # abilita dynamical decoupling
    job = sampler.run([compiled_circuit], shots=1024)
    print(f"Job inviato con ID: {job.job_id()}")
    result = job.result()
    
    
    counts = result[0].data.c_bits.get_counts()

    print("\nRisultati delle misurazioni (fasi):")
    print(counts)

    found_factors=retrieve_factors(counts, n_qubit_phase, N, a)

    
    if not found_factors:
        print(f"\nNon sono stati trovati fattori non banali per N={N} con a={a}.")
        print("Potrebbe essere necessario riprovare con un altro valore di 'a' o aumentare 'n_qubit_phase'.")
    else:
        print(f"\nFattori trovati (potrebbe essere incompleto): {list(found_factors)}")

    return list(found_factors)


N=57
print(f"\n***** Esecuzione per N={N} *****")
t=time.time()
run_shor_generic(N)
print(f"\nTempo impiegato: {time.time()-t}")



***** Esecuzione per N=57 *****
Tentativo con N = 57, scelto a = 2
Qubit per funzione (n): 6
Qubit per fase (t): 13
QUBIT TOTALI RICHIESTI: 19


2025-09-22 17:02:28,108 - INFO - Pass: ContainsInstruction - 0.00787 (ms)
2025-09-22 17:02:28,109 - INFO - Pass: UnitarySynthesis - 0.00715 (ms)
2025-09-22 17:02:28,358 - INFO - Pass: HighLevelSynthesis - 248.97265 (ms)
2025-09-22 17:02:28,382 - INFO - Pass: BasisTranslator - 23.30303 (ms)
2025-09-22 17:02:28,504 - INFO - Pass: ElidePermutations - 106.63104 (ms)
2025-09-22 17:02:28,519 - INFO - Pass: RemoveDiagonalGatesBeforeMeasure - 2.04229 (ms)
2025-09-22 17:02:28,540 - INFO - Pass: RemoveIdentityEquivalent - 20.28322 (ms)
2025-09-22 17:02:28,594 - INFO - Pass: InverseCancellation - 53.50304 (ms)
2025-09-22 17:02:28,595 - INFO - Pass: ContractIdleWiresInControlFlow - 0.00501 (ms)
2025-09-22 17:02:29,553 - INFO - Pass: CommutativeCancellation - 958.29606 (ms)
2025-09-22 17:02:30,224 - INFO - Pass: ConsolidateBlocks - 670.19796 (ms)
2025-09-22 17:02:30,238 - INFO - Pass: Split2QUnitaries - 13.68332 (ms)
2025-09-22 17:02:30,239 - INFO - Pass: SetLayout - 0.00215 (ms)
2025-09-22 17:02:3

Transpilazione completata.


base_primitive._run:INFO:2025-09-22 17:10:24,909: Submitting job using options {'options': {}, 'version': 2, 'support_qiskit': True}


Job inviato con ID: d38mdq8itjus73f7eflg

Risultati delle misurazioni (fasi):
{'0011000011101': 3, '0010011011111': 3, '0100100000001': 7, '1101111101001': 2, '0001000001100': 1, '0000101110001': 4, '0100111101001': 6, '0100110000100': 5, '0100110010000': 6, '0100101111001': 4, '0110101100001': 4, '0100010111101': 7, '1110001011100': 2, '0100100011100': 5, '1000000111000': 4, '1000101010000': 13, '1011110110000': 1, '0001011001100': 2, '0101111001001': 2, '0110100001100': 2, '1010100000001': 4, '1111111001100': 2, '1010110001111': 1, '0010101000001': 3, '0110111001100': 2, '0101100001101': 2, '1100110110010': 2, '0011100000011': 2, '1100110010010': 8, '1110111001010': 4, '0000110111010': 4, '1000000001011': 3, '0111000000000': 3, '0101100010111': 2, '0000111100100': 3, '0110011010111': 1, '0100001100011': 4, '0010000110010': 4, '1111000110010': 3, '0101100110011': 2, '0100011110010': 7, '1111101101011': 2, '1000001100011': 2, '0110011101101': 4, '0001000011110': 1, '0110011000110': 3, 